In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image

In [6]:
import torchvision.models as models

from torchvision import transforms

In [7]:
## set SEED

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## CREATE MODEL CLASS WITH NECESSARY CONVALUTIONAL LAYERS, POOLING, ACTIVATION AND INNER LAYERS AND OUTPUT LAYER

In [26]:
# --- 1. Define the CNN Model Class (nn.Module) ---
# Input shape: (3, 200, 200) [1]

#import torch.nn as nn

class HairClassifierCNN(nn.Module):
    def __init__(self):
        super(HairClassifierCNN, self).__init__()

        # Layer 1: Convolutional Layer
        # Input Channels: 3 (RGB)
        # Output Channels (filters): 32 [2]
        # Kernel Size: (3, 3), Padding: 0, Stride: 1 [2]
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=(3, 3),
            padding=0,
            stride=1
        )
        self.relu1 = nn.ReLU() # Use 'relu' as activation [2]

        # Layer 2: Max Pooling Layer
        # Pooling Size: (2, 2) [2]
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        # Layer 3: Linear Layer 1
        # 64 neurons, 'relu' activation [2]
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.relu2 = nn.ReLU()

        # Layer 4: Output Layer
        # 1 neuron. Appropriate activation for binary classification (logits) [2]
        self.fc2 = nn.Linear(64, 1)

    ### Model must contain forward function
  ### forward functino specifies how the data will move through the Pytorch model

    def forward(self, x):
            # Convolutional Block
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.pool(x)

            # Flatten the feature map into a vector [2]
            # Starting at dimension 1 (leaving batch size 0 intact)
            x = torch.flatten(x, 1)

            # Fully Connected Block
            x = self.fc1(x)
            x = self.relu2(x)

            # Output (logits for binary classification)
            x = self.fc2(x)
            return x

In [ ]:
### Model must contain forward function
#   ### forward functino specifies how the data will move through the Pytorch model

# def forward(self, x):
#         # Convolutional Block
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool(x)

#         # Flatten the feature map into a vector [2]
#         # Starting at dimension 1 (leaving batch size 0 intact)
#         x = torch.flatten(x, 1)

#         # Fully Connected Block
#         x = self.fc1(x)
#         x = self.relu2(x)

#         # Output (logits for binary classification)
#         x = self.fc2(x)
#         return x

In [28]:
# Instantiate the model and move it to the device
model = HairClassifierCNN()

In [29]:
# Instantiate the Optimizer
# Use torch.optim.SGD with lr=0.002 and momentum=0.8 [3]
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [30]:
### SET MODEL TO device variable (GPU or CPU)

model.to(device)

HairClassifierCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (relu2): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [13]:
### QUESTION 1 WHICH LOSS FUNCTION WILL YOU USE?

# ANSWER = nn.BCEWithLogitsLoss() since it is a Classifier CNN model

In [31]:
## Question 2 ---> What's the total number of parameters of the model?
  ## you can use 'torchsummary' or count manually

  # Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


## Prepare Train and Test dataset of Images

### create class object, load into class, perform all necessary transformations, use DataLoader Class for model

In [43]:
import os
from torch.utils.data import Dataset

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


## Trainsformation to Apply  (per Homework 8)
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
     # ImageNet normalization
])


In [33]:
### LOAD DATA FROM PATH DIRECTORIES AND TRANSFORM IMAGES
  ### TRANSFORMING IMAGES HELPS CREATES THE EQUIVALENT OF FEATURE ENGINEERING TO PROVIDE CNN MODEL EVEN BETTER VIEWS OF THE IMAGES FOR IT TO LEARN

from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

## NOW TRAIN CNN MODEL AND VALIDATE ON TEST DATA

In [ ]:
criterion = nn.BCEWithLogitsLoss()      ## what to calculate your loss on

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.5829, Acc: 0.6866, Val Loss: 0.6919, Val Acc: 0.6368
Epoch 2/10, Loss: 0.5447, Acc: 0.7278, Val Loss: 0.6400, Val Acc: 0.6418
Epoch 3/10, Loss: 0.5020, Acc: 0.7553, Val Loss: 0.7181, Val Acc: 0.6169
Epoch 4/10, Loss: 0.4477, Acc: 0.7915, Val Loss: 0.5874, Val Acc: 0.6866
Epoch 5/10, Loss: 0.3898, Acc: 0.8152, Val Loss: 0.6615, Val Acc: 0.6269
Epoch 6/10, Loss: 0.5372, Acc: 0.7828, Val Loss: 0.7478, Val Acc: 0.6119
Epoch 7/10, Loss: 0.4188, Acc: 0.8252, Val Loss: 2.3148, Val Acc: 0.4826
Epoch 8/10, Loss: 0.6766, Acc: 0.6841, Val Loss: 0.6029, Val Acc: 0.6468
Epoch 9/10, Loss: 0.4882, Acc: 0.7665, Val Loss: 0.6237, Val Acc: 0.6766
Epoch 10/10, Loss: 0.3728, Acc: 0.8402, Val Loss: 0.6190, Val Acc: 0.7114


In [ ]:
## QUESTION 3
## WHAT IS MEDIAN OF TRAINING ACCURACY 
    ### ANSWER = 0.774657

pd.Series(history['acc']).describe()

count    10.000000
mean      0.767541
std       0.054684
min       0.684145
25%       0.734707
50%       0.774657
75%       0.809301
max       0.840200
dtype: float64

In [ ]:
## QUESTION 4
## The Standard Deviation of the Training Loss is 0.093757
pd.Series(history['loss']).describe()

count    10.000000
mean      0.496065
std       0.093757
min       0.372794
25%       0.425990
50%       0.495081
75%       0.542847
max       0.676604
dtype: float64

## Generate More Transformations

In [44]:
import os
from torch.utils.data import Dataset

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


## Trainsformation to Apply  (per Homework 8)
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip()
     # ImageNet normalization
     ])

In [45]:
### LOAD DATA FROM PATH DIRECTORIES AND TRANSFORM IMAGES
  ### TRANSFORMING IMAGES HELPS CREATES THE EQUIVALENT OF FEATURE ENGINEERING TO PROVIDE CNN MODEL EVEN BETTER VIEWS OF THE IMAGES FOR IT TO LEARN

from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='./data/train',
    transform=train_transforms
)

test_dataset = HairDataset(
    data_dir='./data/test',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [46]:
criterion = nn.BCEWithLogitsLoss()      ## what to calculate your loss on

num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.5781, Acc: 0.6954, Val Loss: 0.6397, Val Acc: 0.6219
Epoch 2/10, Loss: 0.5449, Acc: 0.7041, Val Loss: 0.7139, Val Acc: 0.5920
Epoch 3/10, Loss: 0.5574, Acc: 0.7079, Val Loss: 0.5842, Val Acc: 0.7164
Epoch 4/10, Loss: 0.5139, Acc: 0.7416, Val Loss: 0.6912, Val Acc: 0.6418
Epoch 5/10, Loss: 0.4998, Acc: 0.7403, Val Loss: 0.5680, Val Acc: 0.6816
Epoch 6/10, Loss: 0.5176, Acc: 0.7378, Val Loss: 0.8419, Val Acc: 0.5871
Epoch 7/10, Loss: 0.5209, Acc: 0.7366, Val Loss: 0.6824, Val Acc: 0.6368
Epoch 8/10, Loss: 0.5058, Acc: 0.7628, Val Loss: 0.5953, Val Acc: 0.6617
Epoch 9/10, Loss: 0.5108, Acc: 0.7491, Val Loss: 0.5760, Val Acc: 0.6915
Epoch 10/10, Loss: 0.5072, Acc: 0.7478, Val Loss: 0.5887, Val Acc: 0.6667


In [47]:
### Question 5. Mean of test loss for all the epochs (1 point)

## Answer = 

pd.Series(history['val_loss']).mean()

np.float64(0.6481412474501563)

In [48]:
### Question 6. Average of test accuracy for the last 5 epochs (1 point)

## Answer = 

pd.Series(history['val_acc']).mean()

np.float64(0.6497512437810944)